In [150]:
import pandas as pd
import numpy as np
import json
from pathlib import Path  
from ast import literal_eval

In [7]:
pd.set_option('display.max_rows', None)

In [103]:
with open("tools/state_codes.json", 'r') as f:
    state_codes = json.load(f)

with open("tools/countyfy.json", 'r') as f:
    county_dict_raw = json.load(f)

county_dict = {}
for key in county_dict_raw:
    county_dict[literal_eval(key.replace('~', '\\\''))] = county_dict_raw[key]

In [206]:
col_data = "datasets/cost-of-living-database.xlsx"
MIN_COUNTY_SIZE = 25000

In [207]:
col_food = pd.read_excel(col_data, sheet_name="Food Costs")
col_rent = pd.read_excel(col_data, sheet_name="Housing Costs - Rent")

col_rent_temp = col_rent
col_rent_temp.columns = col_rent_temp.iloc[1]
col_rent_temp = col_rent_temp.drop([0,1])
col_rent_temp = col_rent_temp[(col_rent_temp["numadults"] == 1) & 
                              (col_rent_temp["numkids"] == 0) & 
                              ((col_rent_temp["countyortownpop"] >= MIN_COUNTY_SIZE) |
                               (col_rent_temp["stateAbbrev"].isin(["CT", "DC", "ME", "NH", "RI", "VA"])))]
col_rent_temp = col_rent_temp[["stateAbbrev", "countyortownName", "expense.rent", "countyortownpop"]]
# col_rent_temp[]

In [208]:
display(col_rent_temp)

1,stateAbbrev,countyortownName,expense.rent,countyortownpop
2,AL,Autauga County,7145.714278,54571
17,AL,Baldwin County,8069.714278,182265
32,AL,Barbour County,4829.714278,27457
62,AL,Blount County,7949.714278,57322
107,AL,Calhoun County,4949.714278,118572
122,AL,Chambers County,5741.714278,34215
137,AL,Cherokee County,4493.714278,25989
152,AL,Chilton County,5381.714278,43643
182,AL,Clarke County,5249.714278,25833
227,AL,Coffee County,5489.714278,49948


In [209]:
col_rent_temp_test = col_rent_temp

In [210]:
countied = {}

In [211]:
def combineRows(row):
    name = " ".join(row[1].split()[:-1])
    if (name, row[0]) not in county_dict:
#         print((name, row[0]))
        return
    
    if county_dict[(name, row[0])] not in countied:
        countied[county_dict[(name, row[0])]] = {"stateAbbrev": [row[0]], 
                                                 "countyortownName": [county_dict[(name, row[0])]], 
                                                 "expense.rent": [row[2]], 
                                                 "countyortownpop": [0]}
        
    temp = countied[county_dict[(name, row[0])]]
    countied[county_dict[(name, row[0])]]["countyortownpop"][0] += row[3]
    return countied

col_rent_temp_test.apply(combineRows, axis=1)
# col_rent_temp_test

2                                                     None
17                                                    None
32                                                    None
62                                                    None
107                                                   None
122                                                   None
137                                                   None
152                                                   None
182                                                   None
227                                                   None
242                                                   None
287                                                   None
317                                                   None
332                                                   None
347                                                   None
362                                                   None
377                                                   No

In [212]:
countied

{'Fairfield County': {'stateAbbrev': ['CT'],
  'countyortownName': ['Fairfield County'],
  'expense.rent': [13829.7142781554],
  'countyortownpop': [916829]},
 'Hartford County': {'stateAbbrev': ['CT'],
  'countyortownName': ['Hartford County'],
  'expense.rent': [10313.7142781554],
  'countyortownpop': [894014]},
 'Litchfield County': {'stateAbbrev': ['CT'],
  'countyortownName': ['Litchfield County'],
  'expense.rent': [9485.71427815536],
  'countyortownpop': [189927]},
 'Middlesex County': {'stateAbbrev': ['CT'],
  'countyortownName': ['Middlesex County'],
  'expense.rent': [10313.7142781554],
  'countyortownpop': [1199721]},
 'New Haven County': {'stateAbbrev': ['CT'],
  'countyortownName': ['New Haven County'],
  'expense.rent': [12125.7142781554],
  'countyortownpop': [862477]},
 'New London County': {'stateAbbrev': ['CT'],
  'countyortownName': ['New London County'],
  'expense.rent': [9569.71427815536],
  'countyortownpop': [274055]},
 'Tolland County': {'stateAbbrev': ['CT'],


In [213]:
for new_county in countied:
    col_rent_temp = pd.concat([col_rent_temp, pd.DataFrame(countied[new_county])], ignore_index=True)

In [214]:
col_rent = col_rent_temp.loc[col_rent_temp["countyortownpop"] >= MIN_COUNTY_SIZE].sort_values(['stateAbbrev', 'countyortownName']).reset_index().drop(columns=["index"])
col_rent = col_rent.rename(columns={"stateAbbrev": "state", "countyortownName": "county", "countyortownpop": "pop"})

In [218]:
len(col_rent)

1729

In [216]:
col_rent

,state,county,expense.rent,pop
0,AK,Anchorage Municipality,10853.714278,291826
1,AK,Fairbanks North Star Borough,11093.714278,97581
2,AK,Juneau City and Borough,12005.714278,31275
3,AK,Kenai Peninsula Borough,9149.714278,55400
4,AK,Matanuska-Susitna Borough,8573.714278,88995
5,AL,Autauga County,7145.714278,54571
6,AL,Baldwin County,8069.714278,182265
7,AL,Barbour County,4829.714278,27457
8,AL,Blount County,7949.714278,57322
9,AL,Calhoun County,4949.714278,118572


In [219]:
col_rent.to_csv("./dataframes/col_rent.csv")